In [ ]:
import pandas as pd
import matplotlib.colors as mc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import yaml
import pypsa
import calendar

from matplotlib.cm import ScalarMappable
from pypsa.descriptors import Dict
from scipy import stats
import seaborn as sns

from math import radians, cos, sin, asin, sqrt

### Select run & Prepare data

In [ ]:
def load_configuration(config_path):
    """
    Load configuration settings from a YAML file.
    """
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    return config


snakemake = Dict()
snakemake.config = load_configuration("../config.yaml")
snakemake.input = Dict()
snakemake.output = Dict()

In [ ]:
run = "test-newplots-1H-cfe100-spatial-noexcess"  # run name from config.yaml
distance = "IEDK"  # pair name from config.yaml

if True:
    folder = f"/results/{run}"
    scenario = f"/2025/p1/cfe100/{distance}"

    snakemake.input.data = f"{folder}/networks/{scenario}/40.nc"
    snakemake.output.plot = f"{folder}/plots/plot.pdf"

    n = pypsa.Network(f"../{folder}/networks/{scenario}/40.nc")

### Explore the data

In [ ]:
locations = list(snakemake.config["ci"][f"{distance}"]["datacenters"].values())
location0 = locations[0]
location1 = locations[1]
locations

In [ ]:
# display two dataframes containing the time series
df = n.generators_t.p_max_pu.filter(regex=f"{location0}|{location1}")
df["2013-03-01":"2013-03-08"].filter(like="solar").plot()

In [ ]:
df = (
    n.generators_t.p_max_pu.filter(regex=f"{location0}|{location1}")
    .reset_index()
    .rename(columns={"index": "snapshot"})
    .assign(snapshot=lambda x: pd.to_datetime(x["snapshot"]))
)
df

### Visualise time-series of RES feed-in

In [ ]:
def prepare_heatmap_data(df, month, location, carrier, scaling, diff_location=None):
    data = df[df["snapshot"].dt.month == month]
    day = data["snapshot"].dt.day
    value = data[f"{location} {carrier}"].values

    if diff_location:
        diff_value = data[f"{diff_location} {carrier}"].values
        value -= diff_value  # Subtracting the second time-series from the first

    value = value.reshape(int(24 / scaling), len(day.unique()), order="F")
    return day, value


def draw_heatmap(ax, day, value, scaling, colormap, min_val, max_val):
    xgrid = np.arange(day.max() + 1) + 1  # for days
    ygrid = np.arange(int(24 / scaling) + 1)  # for hours

    # Ensure the dimensions of 'value' match the expected dimensions for 'xgrid' and 'ygrid'
    if value.shape != (len(ygrid) - 1, len(xgrid) - 1):
        raise ValueError(
            f"Shape of value ({value.shape}) does not match xgrid ({len(xgrid)}) and ygrid ({len(ygrid)}) dimensions."
        )

    ax.pcolormesh(xgrid, ygrid, value, cmap=colormap, vmin=min_val, vmax=max_val)
    ax.set_ylim(int(24 / scaling), 0)
    ax.axis("off")


def plot_heatmap_cf(
    df,
    location,
    carrier,
    scaling,
    colormap,
    min_val,
    max_val,
    year=2013,
    figsize=(14, 5),
    plot_difference=False,
    diff_location=None,
):
    fig, axes = plt.subplots(1, 12, figsize=figsize, sharey=True)
    plt.tight_layout()

    for month, ax in enumerate(axes, start=1):
        # Pass additional parameters if plotting the difference
        day, value = prepare_heatmap_data(
            df,
            month,
            location,
            carrier,
            scaling,
            diff_location=diff_location if plot_difference else None,
        )
        draw_heatmap(ax, day, value, scaling, colormap, min_val, max_val)
        ax.set_title(calendar.month_abbr[month], fontsize=10, pad=3)

    fig.subplots_adjust(
        left=0.05, right=0.98, top=0.9, hspace=0.08, wspace=0.1, bottom=0.15
    )
    cbar_ax = fig.add_axes([0.3, 0.08, 0.4, 0.04])
    norm = mc.Normalize(min_val, max_val)
    cb = fig.colorbar(
        ScalarMappable(norm=norm, cmap=colormap), cax=cbar_ax, orientation="horizontal"
    )
    cb.set_label("Hourly Capacity Factor (%)", size=14)

    fig.text(0.12, 0.12, "Day of the Month", ha="center", va="center", fontsize=14)
    fig.text(
        0.04,
        0.34,
        "Hour of the Day",
        ha="center",
        va="center",
        rotation="vertical",
        fontsize=14,
    )

    annotations = [
        f"Location: {location}"
        if not plot_difference
        else f"Location: {location} vs {diff_location}",
        f"Carrier: {carrier}",
        f"Weather Year: {year}",
        r"Unit: MWh·h$^{-1}$",
    ]
    for i, annotation in enumerate(annotations):
        fig.text(
            0.95,
            0.12 - i * 0.05,
            annotation,
            ha="right",
            va="center",
            fontsize=14,
            color="black",
        )

    fig.savefig("test.pdf", bbox_inches="tight", transparent=True)

In [ ]:
location = f"{location1}"
carrier = "solar"
scaling = int(snakemake.config["time_sampling"][0])  # temporal scaling -- 3/1 for 3H/1H
colormap = "cividis"  # "cividis" # "RdBu"  # https://matplotlib.org/stable/tutorials/colors/colormaps.html
MIN, MAX = 0, 1  #  df["denmark onwind"].min()

plot_heatmap_cf(
    df,
    location,
    carrier,
    scaling,
    colormap,
    MIN,
    MAX,
    diff_location=f"{location0}",
    plot_difference=False,
)

In [ ]:
location = f"{location0}"
carrier = "onwind"
scaling = int(snakemake.config["time_sampling"][0])  # temporal scaling -- 3/1 for 3H/1H
colormap = "RdBu"  # "cividis" # "RdBu"  # https://matplotlib.org/stable/tutorials/colors/colormaps.html
MIN, MAX = -1, 1  #  df["denmark onwind"].min()

plot_heatmap_cf(
    df,
    location,
    carrier,
    scaling,
    colormap,
    MIN,
    MAX,
    diff_location=f"{location1}",
    plot_difference=True,
)

# safe as pdf to local folder
plt.savefig("onwind.pdf", bbox_inches="tight", transparent=True)

In [ ]:
location = f"{location0}"
carrier = "solar"
scaling = int(snakemake.config["time_sampling"][0])  # temporal scaling -- 3/1 for 3H/1H
colormap = "RdBu"  # "cividis" # "RdBu"  # https://matplotlib.org/stable/tutorials/colors/colormaps.html
MIN, MAX = -1, 1  #  df["denmark onwind"].min()

plot_heatmap_cf(
    df,
    location,
    carrier,
    scaling,
    colormap,
    MIN,
    MAX,
    diff_location=f"{location1}",
    plot_difference=True,
)

# safe as pdf to local folder
plt.savefig("solar.pdf", bbox_inches="tight", transparent=True)

### Retrieve space-time shifts from model solution

In [ ]:
def retrieve_nb(n, node, rename={}):
    """
    Retrieve nodal energy balance per hour
        -> lines and links are bidirectional AND their subsets are exclusive.
        -> links include fossil gens
    NB {-1} multiplier is a nodal balance sign
    """

    components = ["Generator", "Load", "StorageUnit", "Store", "Link", "Line"]
    nodal_balance = pd.DataFrame(index=n.snapshots)

    for i in components:
        if i == "Generator":
            node_generators = n.generators.query("bus==@node").index
            nodal_balance = nodal_balance.join(n.generators_t.p[node_generators])
        if i == "Load":
            node_loads = n.loads.query("bus==@node").index
            nodal_balance = nodal_balance.join(-1 * n.loads_t.p_set[node_loads])
        if i == "Link":
            node_export_links = n.links.query("bus0==@node").index
            node_import_links = n.links.query("bus1==@node").index
            nodal_balance = nodal_balance.join(-1 * n.links_t.p0[node_export_links])
            nodal_balance = nodal_balance.join(-1 * n.links_t.p1[node_import_links])
            ##################
        if i == "StorageUnit":
            # node_storage_units = n.storage_units.query('bus==@node').index
            # nodal_balance = nodal_balance.join(n.storage_units_t.p_dispatch[node_storage_units])
            # nodal_balance = nodal_balance.join(n.storage_units_t.p_store[node_storage_units])
            continue
        if i == "Line":
            continue
        if i == "Store":
            continue

    nodal_balance = nodal_balance.rename(columns=rename).groupby(level=0, axis=1).sum()

    # Custom groupby function
    def custom_groupby(column_name):
        if column_name.startswith("vcc"):
            return "spatial shift"
        return column_name

    # Apply custom groupby function
    nodal_balance = nodal_balance.groupby(custom_groupby, axis=1).sum()

    # revert nodal balance sign for display
    if "spatial shift" in nodal_balance.columns:
        nodal_balance["spatial shift"] = nodal_balance["spatial shift"] * -1
    if "temporal shift" in nodal_balance.columns:
        nodal_balance["temporal shift"] = nodal_balance["temporal shift"] * -1

    return nodal_balance

In [ ]:
def hourly_curtailment(network, tech, buses):
    """
    Calculate the curtailment for a given technology and bus.

    :param network: The PyPSA network object
    :param tech: Technology type (string)
    :param buses: Buses to consider for the calculation (iterable of strings)
    :param weights: Weights for the calculation (pandas Series or similar)
    :return: Total curtailment for the given technology and buses
    """
    weights = n.snapshot_weightings["generators"]
    gens = network.generators.query("carrier == @tech and bus in @buses").index
    curtailment = (
        (
            network.generators_t.p_max_pu[gens] * network.generators.p_nom_opt[gens]
            - network.generators_t.p[gens]
        )
        .clip(lower=0)
        .multiply(weights, axis=0)
        .sum(axis=1)
    )
    return curtailment

In [ ]:
def analyze_datacenter_shifts(n, dc1, dc2):
    """
    Analyze the shifts in energy feed-in and spatial shift for two datacenters.

    :param n: PyPSA Network object
    :param dc1: Name of the first datacenter
    :param dc2: Name of the second datacenter
    :return: Dictionary with analysis results

    NB Positive shift -> sending jobs away; negative shift -> receiving jobs
    """

    # retrieve datacenter data
    def analyze_dc(dc_name):
        feedin = retrieve_nb(n, dc_name)[[f"{dc_name} onwind", f"{dc_name} solar"]]
        curtailment = hourly_curtailment(n, "onwind", [dc_name]) + hourly_curtailment(
            n, "solar", [dc_name]
        )
        spatial_shift = retrieve_nb(n, dc_name)["spatial shift"]

        return {
            "feedin": feedin,
            "curtailment": curtailment,
            "spatial_shift": spatial_shift,
        }

    # Analyze both datacenters
    dc1_analysis = analyze_dc(dc1)
    dc2_analysis = analyze_dc(dc2)

    # Collect and store wind and solar hourly potentials
    potentials_dc1 = n.generators_t.p_max_pu[[f"{dc1} onwind", f"{dc1} solar"]]
    potentials_dc2 = n.generators_t.p_max_pu[[f"{dc2} onwind", f"{dc2} solar"]]

    # Compute differences between wind and solar feed-in
    diff_onwind = (
        dc1_analysis["feedin"][f"{dc1} onwind"]
        - dc2_analysis["feedin"][f"{dc2} onwind"]
    )
    diff_solar = (
        dc1_analysis["feedin"][f"{dc1} solar"] - dc2_analysis["feedin"][f"{dc2} solar"]
    )

    # Compute differences between wind and solar potentials
    diff_onwind_potential = (
        potentials_dc1[f"{dc1} onwind"] - potentials_dc2[f"{dc2} onwind"]
    )
    diff_solar_potential = (
        potentials_dc1[f"{dc1} solar"] - potentials_dc2[f"{dc2} solar"]
    )

    return {
        f"{dc1}": {
            "feedin": dc1_analysis["feedin"],
            "potentials": potentials_dc1,
            "curtailment": dc1_analysis["curtailment"],
            "spatial_shift": dc1_analysis["spatial_shift"],
        },
        f"{dc2}": {
            "feedin": dc2_analysis["feedin"],
            "potentials": potentials_dc2,
            "curtailment": dc2_analysis["curtailment"],
            "spatial_shift": dc2_analysis["spatial_shift"],
        },
        "diff_generation": {"onwind": diff_onwind, "solar": diff_solar},
        "diff_potentials": {
            "onwind": diff_onwind_potential,
            "solar": diff_solar_potential,
        },
    }

In [ ]:
# NB Positive shift -> sending jobs away; negative shift -> receiving jobs

correlation_coefficient, p_value = stats.pearsonr(
    analyze_datacenter_shifts(n, dc1=f"{location0}", dc2=f"{location1}")[
        f"{location0}"
    ]["spatial_shift"],
    -(
        analyze_datacenter_shifts(n, dc1=f"{location0}", dc2=f"{location1}")[
            "diff_generation"
        ]["onwind"]
        + analyze_datacenter_shifts(n, dc1=f"{location0}", dc2=f"{location1}")[
            "diff_generation"
        ]["solar"]
    ),
)
# -analyze_datacenter_shifts(n, dc1=f"{location0}", dc2=f"{location1}")['Ireland']["curtailment"]

print(location0, location1)
print(f"Correlation Coefficient: {round(correlation_coefficient, 3)}")
print(f"p-value: {round(p_value,3)}")

### Other checks

In [ ]:
# Potentials correlation
corr_matrix = np.corrcoef(df[f"{location0} solar"], df[f"{location1} solar"])
corr_matrix

In [ ]:
# Potentials correlation
corr_matrix = np.corrcoef(df[f"{location0} onwind"], df[f"{location1} onwind"])
corr_matrix

In [ ]:
# Generation correlation
corr_matrix = np.corrcoef(
    analyze_datacenter_shifts(n, dc1=f"{location0}", dc2=f"{location1}")[
        f"{location0}"
    ]["feedin"][f"{location0} solar"],
    analyze_datacenter_shifts(n, dc1=f"{location1}", dc2=f"{location0}")[
        f"{location1}"
    ]["feedin"][f"{location1} solar"],
)
corr_matrix

In [ ]:
# Generation correlation
corr_matrix = np.corrcoef(
    analyze_datacenter_shifts(n, dc1=f"{location0}", dc2=f"{location1}")[
        f"{location0}"
    ]["feedin"][f"{location0} onwind"],
    analyze_datacenter_shifts(n, dc1=f"{location1}", dc2=f"{location0}")[
        f"{location1}"
    ]["feedin"][f"{location1} onwind"],
)
corr_matrix

### Cost savings VS distance/corr

In [ ]:
def calculate_distance(n, bus1, bus2):
    """
    Calculate the great circle distance between two buses in a PyPSA network object using the Haversine formula.

    Parameters:
    n (DataFrame): PyPSA network object containing bus coordinates.
    bus1 (str): The ID of the first bus.
    bus2 (str): The ID of the second bus.

    Returns:
    float: The distance between the two buses in kilometers.
    """

    def haversine(lon1, lat1, lon2, lat2):
        """
        Calculate the great circle distance between two points on the earth (specified in decimal degrees)
        """
        # Convert decimal degrees to radians
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
        c = 2 * asin(sqrt(a))
        r = 6371  # Radius of earth in kilometers.
        return c * r

    # Extract the coordinates of the two buses
    lon1, lat1 = n.buses.loc[bus1, ["x", "y"]]
    lon2, lat2 = n.buses.loc[bus2, ["x", "y"]]

    # Calculate the distance using the Haversine formula
    distance_km = haversine(lon1, lat1, lon2, lat2)

    return distance_km

In [ ]:
scenarios = snakemake.config["scenario"]["distance"]
flexibilities = snakemake.config["scenario"]["flexibility"]
flexibilities

In [ ]:
data = []

for scenario in scenarios:
    for flexibility in flexibilities:
        n = pypsa.Network(
            f"../{folder}/networks/2025/p1/cfe100/{scenario}/{flexibility}.nc"
        )

        file_path = f"..{folder}/summaries/2025/p1/cfe100/{scenario}/{flexibility}.yaml"
        with open(file_path, "r") as f:
            summary = yaml.safe_load(f)

            # Check if there is only one location
            if len(summary) == 1:
                location = next(iter(summary))  # Get the single location key
                values = summary[location]
                ci_average_cost = values.get("ci_average_cost", None)
                ci_total_cost = round(values.get("ci_total_cost", None) / 1e6, 1)
                if ci_average_cost is not None:
                    data.append(
                        (
                            scenario,
                            flexibility,
                            location,
                            0,  # Distance is 0 for a single location
                            ci_average_cost,
                            ci_total_cost,
                        )
                    )
            else:
                # If more than one location, calculate distances as before
                for location, values in summary.items():
                    ci_average_cost = values.get("ci_average_cost", None)
                    ci_total_cost = round(values.get("ci_total_cost", None) / 1e6, 1)
                    if ci_average_cost is not None:
                        for other_location in summary:
                            if other_location != location:
                                distance = round(
                                    calculate_distance(n, location, other_location), 1
                                )
                                data.append(
                                    (
                                        scenario,
                                        flexibility,
                                        location,
                                        distance,
                                        ci_average_cost,
                                        ci_total_cost,
                                    )
                                )

# Create a multi-index DataFrame
df = pd.DataFrame(
    data,
    columns=[
        "Scenario",
        "Flexibility",
        "Location",
        "Distance",
        "CI_Average_Cost",
        "CI_Total_Cost",
    ],
)
df.set_index(["Scenario", "Flexibility", "Location"], inplace=True)

# Display the DataFrame
df

In [ ]:
# Reset the index of the dataframe
df_reset = df.reset_index()

# Sum Total Costs across locations for each Scenario and Flexibility
total_costs_df = (
    df_reset.groupby(["Scenario", "Flexibility"])
    .agg(
        {
            "CI_Total_Cost": "sum",
            "Distance": "mean",  # Assuming you want to also average the distance for each scenario and flexibility
        }
    )
    .reset_index()
)

# Calculate Baseline Costs (0% Flexibility) for each Scenario
baseline_costs = total_costs_df[total_costs_df["Flexibility"] == "0"][
    ["Scenario", "CI_Total_Cost"]
].rename(columns={"CI_Total_Cost": "Baseline_Cost"})

# Merge the baseline costs with the total_costs_df
df_with_baseline = total_costs_df.merge(baseline_costs, on="Scenario")

# Calculate Cost Savings for each Scenario and Flexibility compared to Baseline
df_with_baseline["Cost_Savings"] = round(
    100
    * (df_with_baseline["Baseline_Cost"] - df_with_baseline["CI_Total_Cost"])
    / df_with_baseline["Baseline_Cost"],
    2,
)

# Now df_with_baseline contains Scenario, Flexibility, mean Distance, Total Cost, Baseline Cost, and Cost Savings
df_with_baseline

In [ ]:
sns.set(style="ticks")
sns.set_context("talk", rc={"lines.linewidth": 3})


def plot_cost_savings(df):
    """
    Plot cost savings as a function of distance with different hues for each flexibility scenario using Seaborn.

    Parameters:
    df (pandas.DataFrame): DataFrame containing 'Distance', 'Cost_Savings', and 'Flexibility' columns.
    """
    # Create the plot with higher DPI for better quality
    plt.figure(figsize=(8, 6), dpi=120)

    # Create a line plot with different hues for each flexibility scenario
    # Use the same marker for each data point
    lineplot = sns.lineplot(
        x="Distance",
        y="Cost_Savings",
        hue="Flexibility",
        style="Flexibility",
        data=df,
        markers="o",
        dashes=False,
        palette="deep",
    )

    # Set the plot title and labels
    plt.title("Cost savings by datacenter distance \n and share of flexible loads")
    plt.xlabel("Haversine distance between datacenter pair (km)")
    plt.ylabel("Cost Savings (%)")

    # Adjust the legend to show only line markers
    handles, labels = lineplot.get_legend_handles_labels()
    # add % to legend labels
    labels = [f"{label}%" for label in labels]
    plt.legend(
        loc="upper left", fontsize="small", ncol=2, handles=handles[:], labels=labels[:]
    )

    # Display the plot
    plt.show()


# Assuming df_with_baseline is the DataFrame prepared earlier
plot_cost_savings(df_with_baseline)